In [ ]:
# %pip install pandas
# %pip install scikit-learn
# %pip install plotly
# %pip install numpy
# %pip install matplotlib
# %pip install nbformat
# %pip install opencv-python
# %pip install tensorflow
# %pip install wandb
# %pip install imblearn
# %pip install seaborn

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import os
import plotly.express as px
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from imblearn.over_sampling import SMOTE

import wandb
%env "WANDB_NOTEBOOK_NAME" "model_data_augmentation_mobilenetv2.ipynb"

wandb.login()

from wandb.integration.keras import WandbMetricsLogger

In [ ]:
label_df = pd.read_csv(
    "../../../data/processed/combined/combined_label.csv", index_col=False
)
data_dir = "../../../data/processed/combined/img/"

label_df.head()

In [ ]:
def load_data(dir_path, img_size):
    X = []
    y = []

    for index, row in label_df.iterrows():
        image_path = os.path.join(dir_path, row["image_name"])
        img = cv2.imread(image_path)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA).astype("float32")

        X.append(img)
        y.append(row["expression_label"])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [ ]:
IMG_SIZE = (96, 96)

X, y = load_data(data_dir, IMG_SIZE)

In [ ]:
dummies = pd.get_dummies(label_df["expression_label"])
dummies.head()

In [ ]:
y = dummies.values
y

In [ ]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=validation_ratio / (train_ratio + test_ratio)
)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

# Distribution of classes in each set


In [ ]:
labels = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
class_count_in_train_set = np.sum(y_train, axis=0)

fig = px.bar(x=labels, y=class_count_in_train_set, color=labels)

fig.update_layout(
    title="Distribution of each label in train set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [ ]:
class_count_in_validation_set = np.sum(y_valid, axis=0)
fig = px.bar(x=labels, y=class_count_in_validation_set, color=labels)

fig.update_layout(
    title="Distribution of each label in validation set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

In [ ]:
class_count_in_test_set = np.sum(y_test, axis=0)
fig = px.bar(x=labels, y=class_count_in_test_set, color=labels)

fig.update_layout(
    title="Distribution of each label in test set",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

# Oversampling with SMOTE


In [ ]:
# get all dimensions of the resulting X
n_samples, height, width, n_channels = [X_train.shape[index] for index in range(4)]

# reshape X because SMOTE accepts only (n_samples, n_channels*height*weight)-type data
X_train_reshaped = X_train.reshape(n_samples, n_channels * height * width)

In [ ]:
# initialize the SMOTE model
smote = SMOTE(random_state=62)

# perform re-sampling on modified X given y
X_train_smote, y_train_smote = smote.fit_resample(X_train_reshaped, y_train)
X_train_smote = X_train_smote.reshape(len(X_train_smote), 96, 96, 3)

In [ ]:
class_count_in_train_smote_set = np.sum(y_train_smote, axis=0)
fig = px.bar(x=labels, y=class_count_in_train_smote_set, color=labels)

fig.update_layout(
    title="Distribution of each label in train set after SMOTE",
    xaxis_title="Expressions",
    yaxis_title="Count",
    height=600,
    width=800,
)

fig.update_traces(texttemplate="%{y}", textposition="inside")

fig.show()

# Model training with data augmentation


In [ ]:
base_model = MobileNetV2(
    weights="imagenet", include_top=False, input_shape=IMG_SIZE + (3,)
)
base_model.trainable = False

In [ ]:
NUM_CLASSES = 7

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    name="mobileNetV2_with_data_augmentation_epoch30_10_batch64",
    # track hyperparameters and run metadata with wandb.config
    config={
        "architecture": "MobileNetV2",
        "optimizer_1": "adam",
        "optimizer_2": "Adam(learning_rate=1e-5)",
        "loss": "categorical_crossentropy",
        "metric": ["categorical_accuracy"],
        "epoch_frozen": 30,
        "epoch_unfrozen": 10,
        "batch_size": 64,
        "IMG_SIZE": "96, 96",
        "Dense_1": 512,
        "Dense_1_activation": "relu",
        "Dropout": 0.2,
        "Dense_2": 7,
        "Dense_2_activation": "softmax",
    },
)

config = wandb.config

In [ ]:
# augment and normalize/standardize for training set
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    brightness_range=(0.8, 1.2),
    zoom_range=0.1,
    samplewise_center=True,
    samplewise_std_normalization=True,
)

# only normalize/standardize for validation and test set
test_val_datagen = ImageDataGenerator(
    samplewise_center=True,
    samplewise_std_normalization=True,
)


# get batch iterator for training
train_iterator = train_datagen.flow(
    X_train_smote, y_train_smote, batch_size=config["batch_size"]
)
# get batch iterator for validation
val_iterator = test_val_datagen.flow(X_valid, y_valid, batch_size=config["batch_size"])

# get batch iterator for test (use only once for evaluation)
test_iterator = test_val_datagen.flow(X_test, y_test, batch_size=config["batch_size"])

In [ ]:
model.compile(
    loss=config["loss"],
    optimizer=config["optimizer"],
    metrics=config["metric"],
)

history = model.fit(
    train_iterator,
    validation_data=val_iterator,
    epochs=config["epoch_frozen"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)

In [ ]:
base_model.trainable = True

model.compile(
    loss=config["loss"],
    optimizer=Adam(learning_rate=1e-5),
    metrics=config["metric"],
)

history = model.fit(
    train_iterator,
    validation_data=val_iterator,
    epochs=config["epoch_unfrozen"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)

wandb.finish()

In [ ]:
model.save("../../../model/240606_mobilenetv2_augmentation_model.keras")

# Confusion matrix with validation set


In [ ]:
def samplewise_standardization(X):
    return (X - np.mean(X)) / np.std(X)

In [ ]:
X_valid_normalized = samplewise_standardization(X_valid)

y_pred = model.predict(X_valid_normalized)
true_class = tf.argmax(y_valid, 1)
predicted_class = tf.argmax(y_pred, 1)

In [ ]:
cm = confusion_matrix(true_class, predicted_class)

ax = plt.subplot()
sns.heatmap(cm, annot=True, fmt="g", ax=ax)

# labels, title and ticks

ax.set_xlabel("Predicted labels")
ax.set_ylabel("True labels")
ax.set_title("Confusion Matrix")

ax.xaxis.set_ticklabels(
    ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
)

ax.yaxis.set_ticklabels(
    ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
)